# 데이터 수집

### 라이브러리 설치

In [18]:
!pip install selenium
!pip install webdriver-manager


^C
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


### 하이라이트 영상 목록에 접속하여 영상들의 url 수집

In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

# Setup Selenium WebDriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run without GUI
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# URL of the YouTube Playlist
playlist_url = "https://www.youtube.com/playlist?list=PLY8T1xH7hxoqIXXJVUPA91JeYD02oRRFc"

# Open the playlist page
driver.get(playlist_url)
time.sleep(5)  # Wait for the page to load

# Scroll to load all videos
def scroll_to_bottom(driver, pause_time=2):
    last_height = driver.execute_script("return document.documentElement.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(pause_time)
        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

scroll_to_bottom(driver)

# Extract video URLs
video_elements = driver.find_elements(By.CSS_SELECTOR, "a.yt-simple-endpoint.style-scope.ytd-playlist-video-renderer")
video_urls = [video.get_attribute("href") for video in video_elements]

# Print extracted URLs
print(f"Extracted {len(video_urls)} video URLs:")
for url in video_urls:
    print(url)

# Save to a file (optional)
with open("fullham_video_urls.txt", "w", encoding="utf-8") as f:
    for url in video_urls:
        f.write(url + "\n")

# Close the WebDriver
driver.quit()


Extracted 43 video URLs:
https://www.youtube.com/watch?v=G0MfkSUMNCk&list=PLY8T1xH7hxoqIXXJVUPA91JeYD02oRRFc&index=1&pp=iAQB
https://www.youtube.com/watch?v=8l9mVMpAWmA&list=PLY8T1xH7hxoqIXXJVUPA91JeYD02oRRFc&index=2&pp=iAQB
https://www.youtube.com/watch?v=3ZQWuPUzkvg&list=PLY8T1xH7hxoqIXXJVUPA91JeYD02oRRFc&index=3&pp=iAQB
https://www.youtube.com/watch?v=nlaXEW2h19Y&list=PLY8T1xH7hxoqIXXJVUPA91JeYD02oRRFc&index=4&pp=iAQB
https://www.youtube.com/watch?v=WC9NCrEtB6g&list=PLY8T1xH7hxoqIXXJVUPA91JeYD02oRRFc&index=5&pp=iAQB
https://www.youtube.com/watch?v=uyYOFSmxm48&list=PLY8T1xH7hxoqIXXJVUPA91JeYD02oRRFc&index=6&pp=iAQB
https://www.youtube.com/watch?v=SawKeJwTi6Q&list=PLY8T1xH7hxoqIXXJVUPA91JeYD02oRRFc&index=7&pp=iAQB
https://www.youtube.com/watch?v=PeT1pZ01Jrg&list=PLY8T1xH7hxoqIXXJVUPA91JeYD02oRRFc&index=8&pp=iAQB
https://www.youtube.com/watch?v=HFJZme86It4&list=PLY8T1xH7hxoqIXXJVUPA91JeYD02oRRFc&index=9&pp=iAQB
https://www.youtube.com/watch?v=hCb_C3Y7bNE&list=PLY8T1xH7hxoqIXXJVUPA91JeY

### 추출한 url에 접속하여 순서대로 댓글 수집

In [4]:
pip install youtube-comment-downloader

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [14]:
from youtube_comment_downloader import YoutubeCommentDownloader
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import json
import time

# WebDriver 설정 (제목 추출용)
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 댓글 및 제목 다운로드 함수
def download_comments_and_title(video_url):
    downloader = YoutubeCommentDownloader()
    
    # 댓글 수집
    comments = []
    try:
        for comment in downloader.get_comments(video_url):
            comments.append(comment['text'])
    except Exception as e:
        print(f"댓글 수집 실패: {e}")
    
    # 제목 수집
    try:
        driver.get(video_url)
        time.sleep(3)  # 페이지 로드 대기
        title_element = driver.find_element(By.CSS_SELECTOR, "h1.title.style-scope.ytd-video-primary-info-renderer")
        video_title = title_element.text
    except Exception as e:
        video_title = "제목을 가져오지 못했습니다."
        print(f"제목 수집 실패: {e}")
    
    return video_title, comments

# 수집할 영상 URL 리스트
# video_urls = [
#     "https://www.youtube.com/watch?v=VIDEO_ID_1",
#     "https://www.youtube.com/watch?v=VIDEO_ID_2",
#     # 추가 URL...
# ]

# 결과 저장 딕셔너리
all_data = {}

for url in video_urls:
    print(f"수집 중: {url}")
    title, comments = download_comments_and_title(url)
    all_data[url] = {
        "title": title,
        "comments": comments
    }
    print(f"수집 완료: {title} - {len(comments)}개의 댓글")

# WebDriver 종료
driver.quit()

# 데이터를 JSON 파일로 저장
with open("fullham_videos_with_comments.json", "w", encoding="utf-8") as file:
    json.dump(all_data, file, ensure_ascii=False, indent=4)

# 결과 출력 (검증용)
for video_url, data in all_data.items():
    print(f"\n영상 URL: {video_url}")
    print(f"영상 제목: {data['title']}")
    print(f"댓글 (최대 5개):")
    for comment in data['comments'][:5]:  # 최대 5개의 댓글만 출력
        print(f"- {comment}")


수집 중: https://www.youtube.com/watch?v=G0MfkSUMNCk&list=PLY8T1xH7hxoqIXXJVUPA91JeYD02oRRFc&index=1&pp=iAQB
수집 완료:  - 56개의 댓글
수집 중: https://www.youtube.com/watch?v=8l9mVMpAWmA&list=PLY8T1xH7hxoqIXXJVUPA91JeYD02oRRFc&index=2&pp=iAQB
수집 완료:  - 289개의 댓글
수집 중: https://www.youtube.com/watch?v=3ZQWuPUzkvg&list=PLY8T1xH7hxoqIXXJVUPA91JeYD02oRRFc&index=3&pp=iAQB
수집 완료:  - 23개의 댓글
수집 중: https://www.youtube.com/watch?v=nlaXEW2h19Y&list=PLY8T1xH7hxoqIXXJVUPA91JeYD02oRRFc&index=4&pp=iAQB
수집 완료:  - 14개의 댓글
수집 중: https://www.youtube.com/watch?v=WC9NCrEtB6g&list=PLY8T1xH7hxoqIXXJVUPA91JeYD02oRRFc&index=5&pp=iAQB
수집 완료:  - 90개의 댓글
수집 중: https://www.youtube.com/watch?v=uyYOFSmxm48&list=PLY8T1xH7hxoqIXXJVUPA91JeYD02oRRFc&index=6&pp=iAQB
수집 완료:  - 53개의 댓글
수집 중: https://www.youtube.com/watch?v=SawKeJwTi6Q&list=PLY8T1xH7hxoqIXXJVUPA91JeYD02oRRFc&index=7&pp=iAQB
수집 완료:  - 7개의 댓글
수집 중: https://www.youtube.com/watch?v=PeT1pZ01Jrg&list=PLY8T1xH7hxoqIXXJVUPA91JeYD02oRRFc&index=8&pp=iAQB
수집 완료:  - 33개의 댓글
수집 중: ht

### 영상의 제목이 제대로 수집되지 않아서 제목 수집

In [16]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

# WebDriver 설정
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # 브라우저 없이 실행
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 플레이리스트 URL
playlist_url = "https://www.youtube.com/playlist?list=PLY8T1xH7hxoqIXXJVUPA91JeYD02oRRFc"

# 플레이리스트 페이지 열기
driver.get(playlist_url)
time.sleep(5)  # 페이지 로드 대기

# 스크롤을 통해 모든 영상 로드
def scroll_to_bottom(driver, pause_time=2):
    last_height = driver.execute_script("return document.documentElement.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(pause_time)
        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

scroll_to_bottom(driver)

# 제목 추출
video_elements = driver.find_elements(By.CSS_SELECTOR, "a.yt-simple-endpoint.style-scope.ytd-playlist-video-renderer")
video_titles = [video.text for video in video_elements if video.text.strip()]

# 결과 출력
print(f"추출된 영상 제목 ({len(video_titles)}개):")
for idx, title in enumerate(video_titles, start=1):
    print(f"{idx}. {title}")

# 드라이버 종료
driver.quit()

# 제목을 파일로 저장
with open("fulham_titles.txt", "w", encoding="utf-8") as f:
    for title in video_titles:
        f.write(title + "\n")


추출된 영상 제목 (43개):
1. HIGHLIGHTS | Luton 2-4 Fulham | Ending The Season In Style 🔥
2. HIGHLIGHTS | Fulham 0-4 Manchester City | Defeat In Final Home Game Of Season 🏡
3. HIGHLIGHTS | Brentford 0-0 Fulham | West London Stalemate
4. HIGHLIGHTS | Fulham 1-1 Palace | Pegged Back At The Death 😤
5. HIGHLIGHTS | Fulham 1-3 Liverpool | Home Defeat Against Title Chasers 😞
6. HIGHLIGHTS | West Ham 0-2 Fulham | Andreas At The Double 🇧🇷
7. HIGHLIGHTS | Nottingham Forest 3-1 Fulham | Defeat At The City Ground 😞
8. HIGHLIGHTS | Sheffield United 3-3 Fulham | Rodrigo acrobatics secure a point 🚲
9. HIGHLIGHTS | Fulham 3-0 Spurs | London Derby Delight 🔥
10. HIGHLIGHTS | Wolves 2-1 Fulham | Tough Loss On The Road
11. HIGHLIGHTS | Fulham 3-0 Brighton | Convincing Win At The Cottage 🙌
12. HIGHLIGHTS | Man Utd 1-2 Fulham | Big Win At Old Trafford 🔥
13. HIGHLIGHTS | Fulham 1-2 Aston Villa | Narrowly Beaten In Battling Display
14. HIGHLIGHTS | Burnley 2-2 Fulham | Late Drama Ends In Draw
15. HIGHLIGHTS | Fulham 

### 제목과 댓글 합침

In [17]:
import json

# 파일 경로 설정
titles_file_path = "fulham_titles.txt"
json_file_path = "fulham_videos_with_comments.json"
updated_json_path = "fulham_videos_with_updated_titles.json"

# 제목 파일 읽기
with open(titles_file_path, "r", encoding="utf-8") as titles_file:
    titles = titles_file.readlines()

# JSON 데이터 로드
with open(json_file_path, "r", encoding="utf-8") as json_file:
    video_data = json.load(json_file)

# 제목 매칭
for i, (url, video) in enumerate(video_data.items()):
    if i < len(titles):  # 제목 파일의 줄 수 확인
        video["title"] = titles[i].strip()

# 업데이트된 JSON 파일 저장
with open(updated_json_path, "w", encoding="utf-8") as updated_json_file:
    json.dump(video_data, updated_json_file, ensure_ascii=False, indent=4)

print(f"업데이트된 JSON 파일이 저장되었습니다: {updated_json_path}")


업데이트된 JSON 파일이 저장되었습니다: fulham_videos_with_updated_titles.json


### 경기 결과 csv 파일과 댓글 json 파일 합치기

In [23]:
import json
import pandas as pd

# Load the JSON file
json_file_path = "fulham_videos_with_updated_titles.json"
with open(json_file_path, "r", encoding="utf-8") as json_file:
    video_data = json.load(json_file)

# Remove "url" and keep only "title" and "comments"
cleaned_data = []
for video in video_data.values():
    title = video.get("title", "No Title")
    comments = video.get("comments", [])
    cleaned_data.append({"title": title, "comments": comments})

# Load the CSV file
csv_file_path = "풀럼결과.csv"
csv_data = pd.read_csv(csv_file_path)
print(len(csv_data))
print(len(cleaned_data))
# Ensure the CSV has the correct number of rows
if len(csv_data) != len(cleaned_data):
    raise ValueError("JSON 데이터와 CSV 데이터의 행 수가 일치하지 않습니다.")

# Merge JSON titles and comments into the CSV
merged_data = []
for i, row in csv_data.iterrows():
    json_entry = cleaned_data[i]
    merged_data.append({
        "csv_column": row.to_dict(),  # CSV의 나머지 열들
        "title": json_entry["title"],
        "comments": json_entry["comments"]
    })

# Save the merged data into a new JSON file
output_json_path = "merged_fulham_data.json"
with open(output_json_path, "w", encoding="utf-8") as output_json:
    json.dump(merged_data, output_json, ensure_ascii=False, indent=4)

# Display success message
print(f"Merged data saved to {output_json_path}")


43
43
Merged data saved to merged_fulham_data.json


## 세 팀(웨스트햄, 풀럼, 본머스)의 데이터 합치기

In [27]:
import json

# File paths
files = [
    'merged_bournemouth_data.json',
    'merged_fulham_data.json',
    'merged_westham_data.json'
]

# Read and merge the JSON files
merged_data = []
for file in files:
    with open(file, 'r', encoding='utf-8') as f:  # UTF-8 인코딩 명시
        data = json.load(f)
        merged_data.extend(data)

# Save the merged data to a new JSON file
output_file = 'merged_all_data.json'
with open(output_file, 'w', encoding='utf-8') as f:  # UTF-8 인코딩 명시
    json.dump(merged_data, f, indent=4, ensure_ascii=False)  # ensure_ascii=False로 한글 저장

output_file


'merged_all_data.json'